In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras import layers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications import EfficientNetB0, InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import Adam

2024-05-07 13:05:06.952743: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-07 13:05:06.952797: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-07 13:05:06.954345: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
train_ds, val_ds = image_dataset_from_directory(
    directory='/kaggle/input/emotion-detection-fer/train/',
    labels='inferred',
    label_mode='categorical',
    class_names=None,
    color_mode='rgb',
    batch_size=32,
    image_size=(299, 299),
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset='both',
    verbose=True
)

Found 28709 files belonging to 7 classes.
Using 22968 files for training.
Using 5741 files for validation.


In [4]:
train_ds

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 299, 299, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 7), dtype=tf.float32, name=None))>

In [5]:
test_ds = image_dataset_from_directory(
    directory='/kaggle/input/emotion-detection-fer/test/',
    labels='inferred',
    label_mode='categorical',
    class_names=None,
    color_mode='rgb',
    batch_size=32,
    image_size=(299, 299),
    shuffle=True,
    seed=42,
    verbose=True
)

Found 7178 files belonging to 7 classes.


In [6]:
input_shape=train_ds.element_spec[0].shape[1:]
input_shape

TensorShape([299, 299, 3])

In [7]:
class_names = train_ds.class_names
print(class_names)


['angry', 'disgusted', 'fearful', 'happy', 'neutral', 'sad', 'surprised']


In [8]:
train_ds.element_spec

(TensorSpec(shape=(None, 299, 299, 3), dtype=tf.float32, name=None),
 TensorSpec(shape=(None, 7), dtype=tf.float32, name=None))

In [9]:
for image_batch, labels_batch in val_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break


(32, 299, 299, 3)
(32, 7)


In [10]:
# import matplotlib.pyplot as plt

# plt.figure(figsize=(10, 10))
# for images, labels in train_ds.take(1):
#   for i in range(9):
#     ax = plt.subplot(3, 3, i + 1)
#     plt.imshow(images[i].numpy().astype("uint8"))
#     plt.title(class_names[labels[i]])
#     plt.axis("off")


In [11]:
def preprocess_data(img, label):
    return preprocess_input(img), label

preproc_test_ds = test_ds.map(preprocess_data)
preproc_train_ds = train_ds.map(preprocess_data)
preproc_val_ds = val_ds.map(preprocess_data)

In [26]:
base_model = InceptionV3(weights="imagenet", include_top=False, input_shape=input_shape)
base_model.trainable = False

aug1 = layers.RandomFlip("horizontal")
aug2 = layers.RandomRotation(0.1)
# aug3 = layers.RandomZoom(0.1)

flatten_layer = Flatten()

dense_layer1 = Dense(700, activation='relu')
dropout1 = Dropout(0.25)
dense_layer2 = Dense(250, activation='relu')
dropout2 = Dropout(0.1)
dense_layer3 = Dense(70, activation='relu')
dropout3 = Dropout(0.1)

prediction_layer = Dense(7, activation='softmax')


model = Sequential([
    base_model,
    aug1,
    aug2,
#     aug3,
    flatten_layer,
    dense_layer1,
    dropout1,
#     dense_layer2,
#     dropout2,
    dense_layer3,
#     dropout3,
    prediction_layer
])

In [27]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ inception_v3 (Functional)       │ ?                      │    21,802,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_flip_2 (RandomFlip)      │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_rotation_2               │ ?                      │   0 (unbuilt) │
│ (RandomRotation)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,802,784 (83.17 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 21,802,784 (83.17 MB)

In [28]:
initial_learning_rate = 0.0001
lr_schedule = ExponentialDecay(
    initial_learning_rate,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=True)

optimizer = Adam(learning_rate=lr_schedule)

In [29]:
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [30]:
es = EarlyStopping(monitor = 'val_accuracy', 
                   mode = 'max', 
                   patience = 3, 
                   verbose = 1, 
                   restore_best_weights = True)

history = model.fit(preproc_train_ds, 
                    validation_data=preproc_val_ds, 
                    epochs=100, 
                    callbacks=[es])

Epoch 1/100
718/718 ━━━━━━━━━━━━━━━━━━━━ 182s 235ms/step - accuracy: 0.3151 - loss: 2.0082 - val_accuracy: 0.4640 - val_loss: 1.3813
Epoch 2/100
718/718 ━━━━━━━━━━━━━━━━━━━━ 165s 230ms/step - accuracy: 0.4672 - loss: 1.4229 - val_accuracy: 0.5179 - val_loss: 1.2893
Epoch 3/100
718/718 ━━━━━━━━━━━━━━━━━━━━ 166s 231ms/step - accuracy: 0.5020 - loss: 1.3195 - val_accuracy: 0.5205 - val_loss: 1.2732
Epoch 4/100
718/718 ━━━━━━━━━━━━━━━━━━━━ 167s 232ms/step - accuracy: 0.5328 - loss: 1.2403 - val_accuracy: 0.5419 - val_loss: 1.2214
Epoch 5/100
718/718 ━━━━━━━━━━━━━━━━━━━━ 165s 230ms/step - accuracy: 0.5549 - loss: 1.1831 - val_accuracy: 0.5367 - val_loss: 1.2532
Epoch 6/100
718/718 ━━━━━━━━━━━━━━━━━━━━ 166s 231ms/step - accuracy: 0.5782 - loss: 1.1365 - val_accuracy: 0.5569 - val_loss: 1.2077
Epoch 7/100
718/718 ━━━━━━━━━━━━━━━━━━━━ 165s 230ms/step - accuracy: 0.5918 - loss: 1.0893 - val_accuracy: 0.5537 - val_loss: 1.1883
Epoch 8/100
718/718 ━━━━━━━━━━━━━━━━━━━━ 166s 230ms/step - accuracy: 

In [31]:
model.evaluate(preproc_test_ds)

225/225 ━━━━━━━━━━━━━━━━━━━━ 33s 144ms/step - accuracy: 0.5834 - loss: 1.2911


[1.2709881067276, 0.5866536498069763]